## 1. Load Data.

In [1]:
import requests
import time
import json

def getUrl(url):
    r = requests.get(url)
    return r
def load_data(dataset_id):
    #for dataset_id in [4,4]:
    start_time = time.time()
    url_list = ["http://localhost:7777/api/graph_bundle_info?dataset_id={}".format(dataset_id)]
    r = getUrl(url_list[0])
    duration_time = time.time() - start_time
    #print(duration_time)
    #print(len(r.text))
    receive_obj = json.loads(r.text)
    #print(receive_obj.keys())
    
    if receive_obj["success"] == True:
        graph_obj = receive_obj["graph_obj"]
        #graph_obj = decompress_data(receive_obj["graph_obj"])
        print(graph_obj["common"]["name"], duration_time)
        #print(receive_obj["graph_obj"].keys())
        pass
    else:
        print("Not success dataset_id:{} time:{}".format(dataset_id, duration_time))
    return graph_obj

## 2. Inconsistency Analysis.

In [48]:

def getMajorLabelList(Info):
    max_value = max(Info['train_nodes'])
    label_list = [i for i,j in enumerate(Info['train_nodes']) if j==max_value]
    return label_list
def error_distribution_analysis(ground_truth, model_output):
    total_node_num = len(ground_truth)
    error_num = 0
    
    for i in range(total_node_num):
        gt = ground_truth[i]
        mo = model_output[i]
        if not gt == mo:
            error_num = error_num + 1
    #print("Error Distribution:")
    #print("Error:{} ({}) Correct:{} ({}) Total:{} ".format(error_num, error_num / total_node_num
    #                                                          , total_node_num - error_num, 1- error_num / total_node_num,
    #                                                          total_node_num))
    return [error_num, error_num / total_node_num
                                                              , total_node_num - error_num, 1- error_num / total_node_num,
                                                              total_node_num]
def Inconsistency_error_pattern_analysis(ground_truth, major_label_dict, model_output):
    total_node_num = len(ground_truth)
    second_satisfied_nodes = 0
    third_satisfied_nodes = 0
    for i in range(total_node_num):
        if i in major_label_dict.keys():
            ## Pass the first condition, unique major label.
            ## Judge whether it is inconsistency.
            gt = ground_truth[i]
            mj = major_label_dict[i]
            if not gt == mj:
                ## Pass the second condition, inconsistency.
                second_satisfied_nodes = second_satisfied_nodes + 1
                mo = model_output[i]
                if not gt == mo:
                    ## Pass the third condition, error.
                    third_satisfied_nodes = third_satisfied_nodes + 1
    if second_satisfied_nodes > 0:
        #error_distribution_analysis(ground_truth, model_output)
        #print("Inconsistency Analysis:")
        #print("Error:{} ({}) Correct:{} ({}) Total:{} ".format(third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
        #                                                      , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
        #                                                      second_satisfied_nodes))
        return [third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
                                                              , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
                                                              second_satisfied_nodes]
    else:
        print("Not found.")
        return []
    
def Multiple_inconsistency_error_pattern_analysis(ground_truth, major_label_dict,KFS_major_label_dict, model_output):
    total_node_num = len(ground_truth)
    second_satisfied_nodes = 0
    third_satisfied_nodes = 0
    for i in range(total_node_num):
        if i in major_label_dict.keys() and i in KFS_major_label_dict:
            ## Pass the first condition, unique major label.
            ## Judge whether it is inconsistency.
            gt = ground_truth[i]
            mj = major_label_dict[i]
            mj2 = KFS_major_label_dict[i]
            if (not gt == mj) and (not gt == mj2):
                ## Pass the second condition, inconsistency.
                second_satisfied_nodes = second_satisfied_nodes + 1
                mo = model_output[i]
                if not gt == mo:
                    ## Pass the third condition, error.
                    third_satisfied_nodes = third_satisfied_nodes + 1
    if second_satisfied_nodes > 0:
        error_distribution_analysis(ground_truth, model_output)
        #print("Inconsistency Analysis:")
        #print("Error:{} ({}) Correct:{} ({}) Total:{} ".format(third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
        #                                                      , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
        #                                                      second_satisfied_nodes))
        return [third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
                                                              , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
                                                              second_satisfied_nodes]
    else:
        print("Not found.")
        return []
    #return third_satisfied_nodes, second_satisfied_nodes
    
def Comprehensive_error_pattern_analysis(ground_truth, major_label_dict,KFS_major_label_dict,CGTNGT_list,CGTNGT_threshold,Distance_list, model_output):
    total_node_num = len(ground_truth)
    second_satisfied_nodes = 0
    third_satisfied_nodes = 0
    second_satisfied_nodes_list = []
    for i in range(total_node_num):
        if i in major_label_dict.keys() and i in KFS_major_label_dict:
            ## Pass the first condition, unique major label.
            ## Judge whether it is inconsistency.
            gt = ground_truth[i]
            mj = major_label_dict[i]
            mj2 = KFS_major_label_dict[i]
            CGTNGT = CGTNGT_list[i]
            Distance = Distance_list[i]
            if (not gt == mj) and (not gt == mj2) and CGTNGT <=CGTNGT_threshold and Distance<=1:
                ## Pass the second condition, inconsistency.
                second_satisfied_nodes = second_satisfied_nodes + 1
                mo = model_output[i]
                second_satisfied_nodes_list.append(i)
                if not gt == mo:
                    ## Pass the third condition, error.
                    third_satisfied_nodes = third_satisfied_nodes + 1
    if second_satisfied_nodes > 0:
        error_distribution_analysis(ground_truth, model_output)
        print("Inconsistency Analysis:")
        print("Error:{} ({}) Correct:{} ({}) Total:{} ".format(third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
                                                              , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
                                                              second_satisfied_nodes))
        print(second_satisfied_nodes_list)
        return [third_satisfied_nodes, third_satisfied_nodes / second_satisfied_nodes
                                                              , second_satisfied_nodes - third_satisfied_nodes, 1- third_satisfied_nodes / second_satisfied_nodes,
                                                              second_satisfied_nodes]
    else:
        print("Not found.")
        return []
    #return third_satisfied_nodes, second_satisfied_nodes

In [22]:
def constructNeighborSet(graph_in):
    neighbor_set = {}
    senders = graph_in["senders"]
    receivers = graph_in["receivers"]
    for i in range(len(senders)):
        send_node = senders[i]
        if not send_node in neighbor_set:
            neighbor_set[send_node] = []
        neighbor_set[send_node].append(receivers[i])
    return neighbor_set
def constructDegreeList(neighbor_set, total_node_num):
    degree_list = []
    for i in range(total_node_num):
        if i in neighbor_set:
            degree = len(neighbor_set[i])
            degree_list.append(degree)
        else:
            degree_list.append(0)
    return degree_list
def constructCGTNGTList(ground_truth,neighbor_set,total_node_num):
    CGTNGT_list = []
    for i in range(total_node_num):
        if i in neighbor_set:
            neighbors = neighbor_set[i]
            center_label = ground_truth[i]
            total = len(neighbors)
            count = 0
            for neighbor_index in neighbors:
                neighbor_label = ground_truth[neighbor_index]
                if neighbor_label == center_label:
                    count = count + 1
            if total > 0:
                CGTNGT = count / total
            else:
                CGTNGT = 0
        else:
            CGTNGT = 0
        CGTNGT_list.append(CGTNGT)
    return CGTNGT_list
def constructDistanceList(SPD, total_node_num):
    dis_list = []
    for i in range(total_node_num):
        dis = SPD[i]['dis']
        if dis == 'inf':
            dis = -1
        dis_list.append(dis)
    return dis_list

## 2. Analyze Data Package.

In [ ]:
def analyze_data(dataset_id=13):
    graph_obj = load_data(dataset_id)
    dataset_name = graph_obj["common"]["name"]
    common = graph_obj["common"]
    individual = graph_obj["individual"]
    SPD = common["graph_additional_info"]["SPD"]
    total_node_num = len(SPD)
    ground_truth = common["graph_target"]["node_features"]
    GCN_output = individual['GCN']["graph_out"]["node_features"]
    GCNWF_output = individual['GCN_Identity_features']["graph_out"]["node_features"]
    MLP_output = individual["MLP"]["graph_out"]["node_features"]
    KFS = common["graph_additional_info"]["KFS"]
    GCN_name = individual["GCN"]["real_model_name"]
    GCNWF_name = individual['GCN_Identity_features']["real_model_name"]
    MLP_name = individual['MLP']["real_model_name"]
    major_label_dict = {}
    for i in range(len(SPD)):
        label_list = getMajorLabelList(SPD[i])
        if len(label_list) == 1:
            major_label_dict[i] = label_list[0]

    KFS_major_label_dict = {}
    for i in range(len(SPD)):
        label_list = getMajorLabelList(KFS[i])
        if len(label_list) == 1:
            KFS_major_label_dict[i] = label_list[0]
    #table_header = ["Dataset", "Model", "Analysis Type", "Error", "Error Rate", "Correct", "Correct Rate", "Total"]
    results_list = []
    GCN_TOTAL_return_result = error_distribution_analysis(ground_truth, GCN_output)
    results_list.append([dataset_name, GCN_name, "TOTAL"] + GCN_TOTAL_return_result)
    GCNWF_TOTAL_return_result = error_distribution_analysis(ground_truth, GCNWF_output)
    results_list.append([dataset_name, GCNWF_name, "TOTAL"] + GCNWF_TOTAL_return_result)
    MLP_TOTAL_return_result = error_distribution_analysis(ground_truth, MLP_output)
    results_list.append([dataset_name, MLP_name, "TOTAL"] + MLP_TOTAL_return_result)
    
    GCN_SPD_return_result = Inconsistency_error_pattern_analysis(ground_truth, major_label_dict, GCN_output)
    results_list.append([dataset_name, GCN_name, "SPD"] + GCN_SPD_return_result)
    GCNWF_SPD_return_result = Inconsistency_error_pattern_analysis(ground_truth, major_label_dict, GCNWF_output)
    results_list.append([dataset_name, GCNWF_name, "SPD"] + GCNWF_SPD_return_result)
    MLP_SPD_return_result = Inconsistency_error_pattern_analysis(ground_truth, major_label_dict, MLP_output)
    results_list.append([dataset_name, MLP_name, "SPD"] + MLP_SPD_return_result)
    
    GCN_KFS_return_result = Inconsistency_error_pattern_analysis(ground_truth, KFS_major_label_dict, GCN_output)
    results_list.append([dataset_name, GCN_name, "KFS"] + GCN_KFS_return_result)
    GCNWF_KFS_return_result = Inconsistency_error_pattern_analysis(ground_truth, KFS_major_label_dict, GCNWF_output)
    results_list.append([dataset_name, GCNWF_name, "KFS"] + GCNWF_KFS_return_result)
    MLP_KFS_return_result = Inconsistency_error_pattern_analysis(ground_truth, KFS_major_label_dict, MLP_output)
    results_list.append([dataset_name, MLP_name, "KFS"] + MLP_KFS_return_result)
    
    GCN_SK_return_result = Multiple_inconsistency_error_pattern_analysis(ground_truth, major_label_dict, KFS_major_label_dict, GCN_output)
    results_list.append([dataset_name, GCN_name, "SPD+KFS"] + GCN_SK_return_result)
    GCNWF_SK_return_result = Multiple_inconsistency_error_pattern_analysis(ground_truth, major_label_dict, KFS_major_label_dict, GCNWF_output)
    results_list.append([dataset_name, GCNWF_name, "SPD+KFS"] + GCNWF_SK_return_result)
    MLP_SK_return_result = Multiple_inconsistency_error_pattern_analysis(ground_truth, major_label_dict, KFS_major_label_dict, MLP_output)
    results_list.append([dataset_name, MLP_name, "SPD+KFS"] + MLP_SK_return_result)
    return results_list


In [ ]:
import csv
with open('outputs/results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    table_header = ["Dataset", "Model", "Analysis Type", "Error", "Error Rate", "Correct", "Correct Rate", "Total"]
    writer.writerow(table_header)
    for dataset_id in [4,5,6,7,9,13]:
        results_list = analyze_data(dataset_id)
        writer.writerows(results_list)
    

In [46]:
def comprehensive_analyze(dataset_id):
    graph_obj = load_data(dataset_id)
    dataset_name = graph_obj["common"]["name"]
    common = graph_obj["common"]
    individual = graph_obj["individual"]
    SPD = common["graph_additional_info"]["SPD"]
    total_node_num = len(SPD)
    ground_truth = common["graph_target"]["node_features"]
    GCN_output = individual['GCN']["graph_out"]["node_features"]
    GCNWF_output = individual['GCN_Identity_features']["graph_out"]["node_features"]
    MLP_output = individual["MLP"]["graph_out"]["node_features"]
    KFS = common["graph_additional_info"]["KFS"]
    GCN_name = individual["GCN"]["real_model_name"]
    GCNWF_name = individual['GCN_Identity_features']["real_model_name"]
    MLP_name = individual['MLP']["real_model_name"]
    major_label_dict = {}
    for i in range(len(SPD)):
        label_list = getMajorLabelList(SPD[i])
        if len(label_list) == 1:
            major_label_dict[i] = label_list[0]

    KFS_major_label_dict = {}
    for i in range(len(SPD)):
        label_list = getMajorLabelList(KFS[i])
        if len(label_list) == 1:
            KFS_major_label_dict[i] = label_list[0]
    graph_in = common["graph_in"]
    neighbor_set = constructNeighborSet(graph_in)
    degree_list = constructDegreeList(neighbor_set, total_node_num)
    CGTNGT_list = constructCGTNGTList(ground_truth, neighbor_set, total_node_num)
    Distance_list = constructDistanceList(SPD, total_node_num)
    GCN_Comp_return_result = Comprehensive_error_pattern_analysis(ground_truth, major_label_dict, KFS_major_label_dict,CGTNGT_list,0.38,Distance_list, GCN_output)
    

In [49]:
for dataset_id in [4,5,6,7,9,13]:
    comprehensive_analyze(dataset_id)

cora 0.19897747039794922
Inconsistency Analysis:
Error:25 (0.9615384615384616) Correct:1 (0.038461538461538436) Total:26 
[162, 323, 332, 498, 554, 606, 644, 767, 1072, 1288, 1329, 1343, 1394, 1494, 1507, 1594, 1649, 1661, 1920, 1983, 1998, 2014, 2015, 2175, 2355, 2427]
citeseer 0.2728714942932129
Inconsistency Analysis:
Error:23 (1.0) Correct:0 (0.0) Total:23 
[155, 367, 487, 653, 825, 1007, 1026, 1096, 1132, 1178, 1338, 1429, 1447, 1819, 1942, 2089, 2119, 2150, 2313, 2529, 2643, 2992, 3197]
pubmed 2.162975788116455
Inconsistency Analysis:
Error:29 (0.9666666666666667) Correct:1 (0.033333333333333326) Total:30 
[799, 863, 2337, 4058, 4129, 4509, 5993, 6424, 6460, 6509, 6572, 6948, 7307, 8165, 8335, 8661, 9767, 10175, 11084, 11134, 11762, 12910, 13054, 15057, 15654, 16446, 18125, 18546, 18813, 19211]
cora_ml 0.3611903190612793
Inconsistency Analysis:
Error:33 (0.7674418604651163) Correct:10 (0.2325581395348837) Total:43 
[315, 353, 522, 692, 756, 770, 930, 1092, 1112, 1141, 1146, 1422,

In [ ]:
Cora-GCN
CGTNGT<=0.0 and Distance in [1.2]
Error:27 (0.9) Correct:3 (0.09999999999999998) Total:30 
CGTNGT<=0.6 and Distance in [1,2]
Error:97 (0.8738738738738738) Correct:14 (0.12612612612612617) Total:111 
CGTNGT<=0.6 and Remove Distance
Error:124 (0.8493150684931506) Correct:22 (0.15068493150684936) Total:146 
Distance<=1  
Error:45 (0.8490566037735849) Correct:8 (0.15094339622641506) Total:53 
CGTNGT<=0.3 and Distance<=1 and SPD inconsistency and KFS inconsistency:
Error:19 (0.95) Correct:1 (0.050000000000000044) Total:20 
CGTNGT<=0.38 and Distance<=1 and SPD+KFS inconsistency
Error:25 (0.9615384615384616) Correct:1 (0.038461538461538436) Total:26 
